In [57]:
import duckdb

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [58]:
%load_ext sql
conn = duckdb.connect(database="data/scd_2.duckdb")

%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [69]:
%%sql
drop table if exists delta;
drop table if exists scd;

,Success


In [60]:

%%sql
/* delta table creation */
create or replace table delta as (
  select 1 id, 190 val, timestamp'2022-06-15' fact_ts union all
  select 1 id, 180 val, timestamp'2022-06-01' fact_ts union all
  select 1, 18, timestamp'2022-06-01' union all        -- dup scd
  select 1, 176, timestamp'2022-05-30' union all
  select 1, 174, timestamp'2022-05-30' union all        -- dup delta
  select 1, 172, timestamp'2022-05-28' union all
  select 1, 16, timestamp'2022-05-24' union all         -- dup scd
  select 1, 150, timestamp'2022-05-20' union all
  select 1, 15, timestamp'2022-05-15' union all         -- dup scd
  select 1, 145, timestamp'2022-05-13' union all
  select 1, 140, timestamp'2022-05-01' union all
  select 1, 120, timestamp'2022-04-01' union all
  select 1, 12, timestamp'2022-04-01' union all
  select 1, 100, timestamp'2022-03-01'
);


,Count
0,14


In [61]:
%%sql
# initial scd creation
create or replace table scd as (
select 1 id, 18 val, date'2022-06-01' effective_from, date'9999-12-31' effective_to, timestamp'2023-01-01' snapshot_update_ts union all
select 1, 17 val, date'2022-05-26', date'2022-05-31', timestamp'2023-01-01' union all
select 1, 16 val, date'2022-05-24', date'2022-05-25', timestamp'2023-01-01' union all
select 1, 15 val, date'2022-05-15', date'2022-05-23', timestamp'2023-01-01' union all
select 1, 14 val, date'2022-05-01', date'2022-05-14', timestamp'2023-01-01' union all
select 1, 13 val, date'2022-04-15', date'2022-04-30', timestamp'2023-01-01' union all
select 1, 12 val, date'2022-04-01', date'2022-04-14', timestamp'2023-01-01' union all
select 1, 11 val, date'2022-03-15', date'2022-03-31', timestamp'2023-01-01' union all
select 2, 22, date'2022-01-15', date'9999-12-31', timestamp'2023-01-01' union all
select 3, 33, date'2022-01-15', date'9999-12-31', timestamp'2023-01-01'
);

,Count
0,10


1063 min - 1M  
 106 min - 10M  
  35 min - 30M  
  21 min - 50M  
  11 min - 100M  
   4 min - 300M  
   2 min - 500M  
   1 min - 1B  

In [63]:
%%sql
-- client 1 history fulfilling
insert into scd
select 1, random()*10000+1, v::timestamp, v::timestamp + interval 106 minute - interval 1 second, timestamp'2023-11-11 23:11:11'
from generate_series(timestamp'2022-03-15' - interval 106 minute, timestamp'1-1-1', interval '-106 minutes') t(v);

,Count
0,10028757


In [64]:
%%sql
-- client 2,3 history fulfilling
insert into scd
select 2, random()*10000+1, v::timestamp, v::timestamp + interval 106 minute - interval 1 second, timestamp'2023-11-11 23:11:12'
from generate_series(timestamp'2022-01-15' - interval 106 minute, timestamp'1-1-1', interval '-106 minutes') t(v);

insert into scd
select 3, random()*10000+1, v::timestamp, v::timestamp + interval 106 minute - interval 1 second, timestamp'2023-11-11 23:11:13'
from generate_series(timestamp'2022-01-15' - interval 106 minute, timestamp'1-1-1', interval '-106 minutes') t(v);

,Count
0,10027956


In [65]:
%%sql
-- SCD: option 1-2. full recalc + dedupe delta only. NOT AN OPTION IN GENERAL

-- explain analyze
-- create or replace table scd as
SELECT COUNT(1) FROM (
  WITH combine_dedupe AS (
    SELECT *
    FROM delta
    QUALIFY ROW_NUMBER() OVER (
      PARTITION BY sha1(to_json((id, fact_ts)))
      ORDER BY fact_ts
    ) = 1

    UNION DISTINCT

    SELECT * EXCLUDE (effective_to, snapshot_update_ts)
    FROM scd
  )

  SELECT
    id,
    val,
    fact_ts AS effective_from,
    IFNULL(
      LEAD(fact_ts) OVER (
        PARTITION BY sha1(to_json((id)))
        ORDER BY fact_ts
      ) - INTERVAL 1 SECOND,
      DATE '9999-12-31 23:59:59'
    ) AS effective_to,
    CURRENT_TIMESTAMP AS snapshot_update_ts
  FROM combine_dedupe
  -- ORDER BY 1, 3 DESC
);
-- create scd


,count(1)


In [66]:
%%sql
with
range as (
  select id, min(fact_ts) min_ts, max(fact_ts) max_ts
  from delta group by all
)
,
combine as (
  select * from delta
  union by name
  select s.* exclude (snapshot_update_ts) rename (effective_from as fact_ts, effective_to as old_effective_to)
  from scd s
  join range r on s.id = r.id and s.effective_to >= r.min_ts
  qualify row_number() over(partition by s.id order by if(s.effective_from > r.max_ts, 1, 0) desc, s.effective_from) = 1
    or s.effective_from <= r.max_ts
)
,
dedupe as (
  select * from combine
  qualify row_number() over (partition by sha1(to_json((
    id, fact_ts
  ))) order by fact_ts, old_effective_to) = 1
)

select * exclude(old_effective_to) rename(fact_ts as effective_from),
  ifnull(lead(fact_ts) over (partition by id order by fact_ts) - interval 1 second, timestamp'9999-12-31 23:59:59') as effective_to,
  current_timestamp as snapshot_update_ts
from dedupe
qualify effective_to <= ifnull(old_effective_to, timestamp'9999-12-31 23:59:59')
order by 1, 3 desc

,id,val,effective_from,effective_to,snapshot_update_ts
0,1,190,2022-06-15,9999-12-31 23:59:59,2025-07-20 16:16:20.241000+03:00
1,1,18,2022-06-01,2022-06-14 23:59:59,2025-07-20 16:16:20.241000+03:00
2,1,176,2022-05-30,2022-05-31 23:59:59,2025-07-20 16:16:20.241000+03:00
3,1,172,2022-05-28,2022-05-29 23:59:59,2025-07-20 16:16:20.241000+03:00
4,1,17,2022-05-26,2022-05-27 23:59:59,2025-07-20 16:16:20.241000+03:00
5,1,150,2022-05-20,2022-05-23 23:59:59,2025-07-20 16:16:20.241000+03:00
6,1,15,2022-05-15,2022-05-19 23:59:59,2025-07-20 16:16:20.241000+03:00
7,1,145,2022-05-13,2022-05-14 23:59:59,2025-07-20 16:16:20.241000+03:00
8,1,14,2022-05-01,2022-05-12 23:59:59,2025-07-20 16:16:20.241000+03:00
9,1,4513,2022-02-28,2022-02-28 23:59:59,2025-07-20 16:16:20.241000+03:00


In [56]:
duckdb.close()